<a href="https://colab.research.google.com/github/vinayak2019/gromacs_automation/blob/main/MDAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install the packages
!pip install --upgrade MDAnalysis
!pip install nglview

In [ ]:
# getting the files
!git clone https://github.com/vinayak2019/gromacs_automation.git
!unzip /content/gromacs_automation/SampleData.zip

In [ ]:
# importing the definitions
import MDAnalysis as mda
from MDAnalysis.analysis import rms


In [ ]:
# creating the universe
u = mda.Universe('/content/SampleData/step5.tpr','/content/SampleData/step5.xtc')

In [ ]:
u

In [ ]:
# getting the last step from trajectory
u.trajectory[-1]

In [ ]:
# setting the trajectory to step 0
u.trajectory[0]

In [ ]:
# getting the RMSD
rmsd_analysis = rms.RMSD(u, select='backbone', groupselections=['name CA', 'protein'])
rmsd_analysis.run()

In [ ]:
# saving the RMSD data to a pandas dataframe
import pandas as pd

rmsd_df = pd.DataFrame(rmsd_analysis.results.rmsd[:, 2:],
                       columns=['Backbone', 'C-alphas', 'Protein'],
                       index=rmsd_analysis.results.rmsd[:, 1])
rmsd_df.index.name = 'Time (ps)'
rmsd_df.head()

In [ ]:
# plotting the RMSD data
rmsd_df.plot(title='RMSD')

In [ ]:
# calculating the RMSF
c_alphas = u.select_atoms('protein and name CA')
rmsf_analysis = rms.RMSF(c_alphas)
R = rmsf_analysis.run()

In [ ]:
# plotting the RMSF data
import matplotlib.pyplot as plt
plt.plot(c_alphas.resids, R.results.rmsf)
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')
plt.axvspan(30, 59, zorder=0, alpha=0.2, color='green', label='NMP')
plt.legend()
plt.show()

#### Contacts

In [ ]:
# Set the two groups
sel_lig = 'resname LIG'
lig = u.select_atoms(sel_lig)

sel_pro = 'protein'
protein = u.select_atoms(sel_pro)

In [ ]:
# get the contacts data
from MDAnalysis.analysis import contacts

ca1 = contacts.Contacts(u,
                        select=(sel_lig, sel_pro),
                        refgroup=(lig, protein),
                        radius=4.5,
                        method='hard_cut').run()

In [ ]:
# create a dataframe from the data
ca1_df = pd.DataFrame(ca1.results.timeseries,
                      columns=['Frame',
                               'Contacts from first frame'])
ca1_df.head()

In [ ]:
# get the total number of contacts
n_ref = ca1.initial_contacts[0].sum()
print('There are {} contacts in the reference.'.format(n_ref))

In [ ]:
# get contacts per time frame
n_contacts = ca1.results.timeseries[:, 1] * n_ref
print(n_contacts)

In [ ]:
#plotting the total number of contacts per frame
ca1_df.plot(x='Frame')
plt.ylabel('Fraction of contacts')
plt.show()